# Rekurrente neuronale Netze

Im vorherigen Modul haben wir uns mit reichhaltigen semantischen Repräsentationen von Texten beschäftigt. Die Architektur, die wir verwendet haben, erfasst die aggregierte Bedeutung der Wörter in einem Satz, berücksichtigt jedoch nicht die **Reihenfolge** der Wörter, da die Aggregationsoperation, die den Einbettungen folgt, diese Information aus dem ursprünglichen Text entfernt. Da diese Modelle die Wortreihenfolge nicht darstellen können, sind sie nicht in der Lage, komplexere oder mehrdeutige Aufgaben wie Textgenerierung oder Beantwortung von Fragen zu lösen.

Um die Bedeutung einer Textsequenz zu erfassen, verwenden wir eine neuronale Netzwerkarchitektur namens **rekurrentes neuronales Netz** (RNN). Bei der Verwendung eines RNN führen wir unseren Satz Token für Token durch das Netzwerk, und das Netzwerk erzeugt einen **Zustand**, den wir dann mit dem nächsten Token erneut in das Netzwerk einspeisen.

![Bild, das ein Beispiel für die Generierung eines rekurrenten neuronalen Netzes zeigt.](../../../../../lessons/5-NLP/16-RNN/images/rnn.png)

Angenommen, wir haben eine Eingabesequenz von Tokens $X_0,\dots,X_n$, dann erstellt das RNN eine Sequenz von neuronalen Netzwerkblöcken und trainiert diese Sequenz end-to-end mithilfe von Backpropagation. Jeder Netzwerkblock nimmt ein Paar $(X_i,S_i)$ als Eingabe und erzeugt $S_{i+1}$ als Ergebnis. Der finale Zustand $S_n$ oder die Ausgabe $Y_n$ wird in einen linearen Klassifikator eingespeist, um das Ergebnis zu erzeugen. Alle Netzwerkblöcke teilen sich die gleichen Gewichte und werden in einem einzigen Backpropagation-Durchlauf end-to-end trainiert.

> Die obige Abbildung zeigt ein rekurrentes neuronales Netz in entfalteter Form (links) und in kompakter rekurrenter Darstellung (rechts). Es ist wichtig zu verstehen, dass alle RNN-Zellen die gleichen **teilbaren Gewichte** haben.

Da Zustandsvektoren $S_0,\dots,S_n$ durch das Netzwerk weitergegeben werden, kann das RNN sequentielle Abhängigkeiten zwischen Wörtern lernen. Zum Beispiel kann es, wenn das Wort *nicht* irgendwo in der Sequenz erscheint, lernen, bestimmte Elemente innerhalb des Zustandsvektors zu negieren.

Innerhalb jeder RNN-Zelle befinden sich zwei Gewichtsmatrizen: $W_H$ und $W_I$, sowie ein Bias $b$. Bei jedem RNN-Schritt wird der Ausgabestatus wie folgt berechnet: $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, wobei $f$ eine Aktivierungsfunktion ist (oft $\tanh$).

> Bei Problemen wie der Textgenerierung (die wir im nächsten Abschnitt behandeln werden) oder der maschinellen Übersetzung möchten wir auch bei jedem RNN-Schritt einen Ausgabewert erhalten. In diesem Fall gibt es eine weitere Matrix $W_O$, und die Ausgabe wird berechnet als $Y_i=f(W_O\times S_i+b_O)$.

Schauen wir uns an, wie rekurrente neuronale Netze uns dabei helfen können, unser Nachrichten-Dataset zu klassifizieren.

> Für die Sandbox-Umgebung müssen wir die folgende Zelle ausführen, um sicherzustellen, dass die erforderliche Bibliothek installiert ist und die Daten vorab geladen werden. Wenn Sie lokal arbeiten, können Sie die folgende Zelle überspringen.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Beim Training großer Modelle kann die Zuweisung von GPU-Speicher problematisch werden. Außerdem müssen wir möglicherweise mit verschiedenen Minibatch-Größen experimentieren, damit die Daten in den GPU-Speicher passen und das Training dennoch schnell genug ist. Wenn Sie diesen Code auf Ihrer eigenen GPU-Maschine ausführen, können Sie mit der Anpassung der Minibatch-Größe experimentieren, um das Training zu beschleunigen.

> **Hinweis**: Es ist bekannt, dass bestimmte Versionen von NVidia-Treibern den Speicher nach dem Training des Modells nicht freigeben. In diesem Notebook führen wir mehrere Beispiele aus, was dazu führen kann, dass der Speicher in bestimmten Konfigurationen erschöpft wird, insbesondere wenn Sie eigene Experimente im selben Notebook durchführen. Wenn Sie auf seltsame Fehler stoßen, wenn Sie mit dem Training des Modells beginnen, sollten Sie den Notebook-Kernel neu starten.


In [3]:
batch_size = 16
embed_size = 64

## Einfacher RNN-Klassifikator

Bei einem einfachen RNN ist jede rekurrente Einheit ein einfaches lineares Netzwerk, das einen Eingabevektor und einen Zustandsvektor aufnimmt und einen neuen Zustandsvektor erzeugt. In Keras kann dies durch die `SimpleRNN`-Schicht dargestellt werden.

Obwohl wir der RNN-Schicht direkt one-hot-codierte Tokens übergeben könnten, ist dies aufgrund ihrer hohen Dimensionalität keine gute Idee. Daher verwenden wir eine Embedding-Schicht, um die Dimensionalität der Wortvektoren zu reduzieren, gefolgt von einer RNN-Schicht und schließlich einem `Dense`-Klassifikator.

> **Hinweis**: In Fällen, in denen die Dimensionalität nicht so hoch ist, wie beispielsweise bei der Tokenisierung auf Zeichenebene, könnte es sinnvoll sein, one-hot-codierte Tokens direkt in die RNN-Zelle einzuspeisen.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Hinweis:** Hier verwenden wir eine untrainierte Einbettungsschicht zur Vereinfachung, aber für bessere Ergebnisse können wir eine vortrainierte Einbettungsschicht mit Word2Vec verwenden, wie im vorherigen Abschnitt beschrieben. Es wäre eine gute Übung für dich, diesen Code so anzupassen, dass er mit vortrainierten Einbettungen funktioniert.

Nun lassen wir unser RNN trainieren. RNNs sind im Allgemeinen recht schwierig zu trainieren, da die Anzahl der Schichten, die bei der Rückpropagation beteiligt sind, sehr groß wird, sobald die RNN-Zellen entlang der Sequenzlänge entfaltet werden. Daher müssen wir eine kleinere Lernrate wählen und das Netzwerk auf einem größeren Datensatz trainieren, um gute Ergebnisse zu erzielen. Dies kann ziemlich lange dauern, daher wird die Verwendung einer GPU empfohlen.

Um den Prozess zu beschleunigen, werden wir das RNN-Modell nur mit Nachrichtentiteln trainieren und die Beschreibung weglassen. Du kannst versuchen, mit der Beschreibung zu trainieren und sehen, ob du das Modell zum Laufen bringen kannst.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Hinweis**: Die Genauigkeit ist hier wahrscheinlich geringer, da wir nur mit Nachrichtentiteln trainieren.


## Überprüfung von variablen Sequenzen

Denken Sie daran, dass die `TextVectorization`-Schicht automatisch Sequenzen mit variabler Länge in einem Minibatch mit Pad-Tokens auffüllt. Es stellt sich heraus, dass diese Tokens auch am Training teilnehmen und die Konvergenz des Modells erschweren können.

Es gibt mehrere Ansätze, die wir verfolgen können, um die Menge an Padding zu minimieren. Einer davon ist, den Datensatz nach Sequenzlänge neu zu ordnen und alle Sequenzen nach Größe zu gruppieren. Dies kann mit der Funktion `tf.data.experimental.bucket_by_sequence_length` durchgeführt werden (siehe [Dokumentation](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Ein weiterer Ansatz ist die Verwendung von **Maskierung**. In Keras unterstützen einige Schichten zusätzliche Eingaben, die zeigen, welche Tokens beim Training berücksichtigt werden sollen. Um Maskierung in unser Modell zu integrieren, können wir entweder eine separate `Masking`-Schicht einfügen ([Dokumentation](https://keras.io/api/layers/core_layers/masking/)) oder den Parameter `mask_zero=True` in unserer `Embedding`-Schicht angeben.

> **Note**: Dieses Training wird etwa 5 Minuten dauern, um eine Epoche auf dem gesamten Datensatz abzuschließen. Sie können das Training jederzeit unterbrechen, wenn Sie die Geduld verlieren. Alternativ können Sie die Menge der für das Training verwendeten Daten begrenzen, indem Sie nach den Datensätzen `ds_train` und `ds_test` eine `.take(...)`-Klausel hinzufügen.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Jetzt, da wir Maskierung verwenden, können wir das Modell mit dem gesamten Datensatz aus Titeln und Beschreibungen trainieren.

> **Hinweis**: Ist Ihnen aufgefallen, dass wir einen Vektorisierer verwenden, der auf den Nachrichtentiteln trainiert wurde, und nicht auf dem gesamten Artikeltext? Das könnte dazu führen, dass einige Tokens ignoriert werden. Es wäre daher besser, den Vektorisierer neu zu trainieren. Allerdings könnte der Effekt nur sehr gering sein, daher bleiben wir der Einfachheit halber beim vorher trainierten Vektorisierer.


## LSTM: Langzeit-Kurzzeitspeicher

Eines der Hauptprobleme von RNNs sind **verschwindende Gradienten**. RNNs können ziemlich lang sein und haben möglicherweise Schwierigkeiten, die Gradienten während der Rückwärtspropagation bis zur ersten Schicht des Netzwerks zurückzuführen. Wenn dies passiert, kann das Netzwerk keine Beziehungen zwischen weit entfernten Tokens lernen. Eine Möglichkeit, dieses Problem zu vermeiden, besteht darin, **explizites Zustandsmanagement** durch den Einsatz von **Gates** einzuführen. Die beiden gängigsten Architekturen, die Gates verwenden, sind **Langzeit-Kurzzeitspeicher** (LSTM) und **Gated Relay Unit** (GRU). Hier werden wir uns mit LSTMs beschäftigen.

![Bild, das eine Beispielzelle eines Langzeit-Kurzzeitspeichers zeigt](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Ein LSTM-Netzwerk ist ähnlich wie ein RNN organisiert, aber es gibt zwei Zustände, die von Schicht zu Schicht weitergegeben werden: den tatsächlichen Zustand $c$ und den versteckten Vektor $h$. In jeder Einheit wird der versteckte Vektor $h_{t-1}$ mit der Eingabe $x_t$ kombiniert, und zusammen steuern sie, was mit dem Zustand $c_t$ und der Ausgabe $h_{t}$ durch **Gates** geschieht. Jedes Gate hat eine Sigmoid-Aktivierung (Ausgabe im Bereich $[0,1]$), die als bitweises Maskieren betrachtet werden kann, wenn sie mit dem Zustandsvektor multipliziert wird. LSTMs haben die folgenden Gates (von links nach rechts im obigen Bild):
* **Vergessens-Gate**, das bestimmt, welche Komponenten des Vektors $c_{t-1}$ wir vergessen und welche wir durchlassen müssen.
* **Eingabe-Gate**, das bestimmt, wie viele Informationen aus dem Eingabevektor und dem vorherigen versteckten Vektor in den Zustandsvektor aufgenommen werden sollen.
* **Ausgabe-Gate**, das den neuen Zustandsvektor nimmt und entscheidet, welche seiner Komponenten verwendet werden, um den neuen versteckten Vektor $h_t$ zu erzeugen.

Die Komponenten des Zustands $c$ können als Flags betrachtet werden, die ein- und ausgeschaltet werden können. Zum Beispiel, wenn wir im Sequenzverlauf den Namen *Alice* sehen, vermuten wir, dass es sich um eine Frau handelt, und setzen das Flag im Zustand, das anzeigt, dass wir ein weibliches Substantiv im Satz haben. Wenn wir später die Wörter *und Tom* sehen, setzen wir das Flag, das anzeigt, dass wir ein Plural-Substantiv haben. Durch die Manipulation des Zustands können wir also die grammatikalischen Eigenschaften des Satzes verfolgen.

> **Note**: Hier ist eine großartige Ressource, um die Interna von LSTMs zu verstehen: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) von Christopher Olah.

Obwohl die interne Struktur einer LSTM-Zelle komplex erscheinen mag, verbirgt Keras diese Implementierung in der `LSTM`-Schicht. Das Einzige, was wir im obigen Beispiel tun müssen, ist, die rekurrente Schicht zu ersetzen:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## Bidirektionale und mehrschichtige RNNs

In unseren bisherigen Beispielen arbeiten die rekurrenten Netzwerke von Anfang bis Ende einer Sequenz. Das erscheint uns natürlich, da es der Richtung entspricht, in der wir lesen oder Sprache hören. Für Szenarien, die einen zufälligen Zugriff auf die Eingabesequenz erfordern, ist es jedoch sinnvoller, die rekurrente Berechnung in beide Richtungen auszuführen. RNNs, die Berechnungen in beide Richtungen ermöglichen, werden als **bidirektionale** RNNs bezeichnet, und sie können erstellt werden, indem die rekurrente Schicht mit einer speziellen `Bidirectional`-Schicht umwickelt wird.

> **Note**: Die `Bidirectional`-Schicht erstellt zwei Kopien der darin enthaltenen Schicht und setzt die Eigenschaft `go_backwards` einer dieser Kopien auf `True`, sodass sie in die entgegengesetzte Richtung entlang der Sequenz läuft.

Rekurrente Netzwerke, ob unidirektional oder bidirektional, erfassen Muster innerhalb einer Sequenz und speichern sie in Zustandsvektoren oder geben sie als Ausgabe zurück. Wie bei konvolutionalen Netzwerken können wir eine weitere rekurrente Schicht hinzufügen, die der ersten folgt, um höherwertige Muster zu erfassen, die aus niedrigeren Mustern bestehen, die von der ersten Schicht extrahiert wurden. Dies führt uns zum Konzept eines **mehrschichtigen RNN**, das aus zwei oder mehr rekurrenten Netzwerken besteht, wobei die Ausgabe der vorherigen Schicht als Eingabe an die nächste Schicht weitergegeben wird.

![Bild, das ein mehrschichtiges Long-Short-Term-Memory-RNN zeigt](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*Bild aus [diesem großartigen Beitrag](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) von Fernando López.*

Keras macht den Aufbau dieser Netzwerke zu einer einfachen Aufgabe, da Sie einfach weitere rekurrente Schichten zum Modell hinzufügen müssen. Für alle Schichten außer der letzten müssen wir den Parameter `return_sequences=True` angeben, da wir möchten, dass die Schicht alle Zwischenzustände zurückgibt und nicht nur den Endzustand der rekurrenten Berechnung.

Lassen Sie uns ein zweischichtiges bidirektionales LSTM für unser Klassifizierungsproblem erstellen.

> **Note** Dieser Code benötigt erneut ziemlich viel Zeit, um abgeschlossen zu werden, aber er liefert uns die höchste Genauigkeit, die wir bisher gesehen haben. Es könnte sich also lohnen, zu warten und das Ergebnis zu sehen.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNNs für andere Aufgaben

Bis jetzt haben wir uns darauf konzentriert, RNNs zur Klassifikation von Textsequenzen zu verwenden. Aber sie können noch viele weitere Aufgaben bewältigen, wie zum Beispiel Textgenerierung und maschinelle Übersetzung — diese Aufgaben werden wir in der nächsten Einheit betrachten.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
